In [36]:
!pip -q install sagemaker
!pip -q install ipywidgets
!pip install sagemaker[local]


You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
from IPython.display import display
from ipywidgets import Checkbox

box = Checkbox(False, description='Local?')
display(box)

def changed(b):
    print(b)

box.observe(changed)

Checkbox(value=False, description='Local?')

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/hunkim-kogpt2'

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='hunkimSagemaker')['Role']['Arn']
    
print(role)

arn:aws:iam::294038372338:role/hunkimSagemaker


In [3]:
%%bash
#wget http://research.metamind.io.s3.amazonaws.com/wikitext/wikitext-2-raw-v1.zip
wget -O wikitext-2-raw-v1.zip https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
unzip -n wikitext-2-raw-v1.zip
cd wikitext-2-raw
mv wiki.test.raw test && mv wiki.train.raw train && mv wiki.valid.raw valid


Archive:  wikitext-2-raw-v1.zip
  inflating: wikitext-2-raw/wiki.test.raw  
  inflating: wikitext-2-raw/wiki.valid.raw  
  inflating: wikitext-2-raw/wiki.train.raw  
--2020-06-09 16:09:11--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-2-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.207.237
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.207.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4721645 (4.5M) [application/zip]
Saving to: ‘wikitext-2-raw-v1.zip’

     0K .......... .......... .......... .......... ..........  1%  250K 18s
    50K .......... .......... .......... .......... ..........  2%  249K 18s
   100K .......... .......... .......... .......... ..........  3%  250K 18s
   150K .......... .......... .......... .......... ..........  4% 92.1M 13s
   200K .......... .......... .......... .......... ..........  5% 75.0M 10s
   250K .......... .......... .......... .......... ..........  6%  251K 12

In [4]:
!head -5 wikitext-2-raw/train


 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : Unrecorded Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series n

In [5]:
inputs = sagemaker_session.upload_data(path='wikitext-2-raw', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-west-2-294038372338/sagemaker/hunkim-kogpt2


In [6]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='train.py',
                    source_dir='code',
                    role=role,
                    framework_version='1.5.0',
                    train_instance_count=1,
                    #train_instance_type='ml.p2.xlarge',
                    train_instance_type='local',

                    # available hyperparameters: emsize, nhid, nlayers, lr, clip, epochs, batch_size,
                    #                            bptt, dropout, tied, seed, log_interval
                    hyperparameters={
                        'epochs': 1,
                        'tied': True
                    })

In [8]:
estimator.fit({'training': inputs})

Creating tmpu1ya9mng_algo-1-i0u2q_1 ... 
Attaching to tmpu1ya9mng_algo-1-i0u2q_1
algo-1-i0u2q_1  | 2020-06-09 07:11:57,679 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-i0u2q_1  | 2020-06-09 07:11:57,692 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-i0u2q_1  | 2020-06-09 07:11:57,705 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-i0u2q_1  | 2020-06-09 07:11:57,710 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-i0u2q_1  | 2020-06-09 07:11:58,561 sagemaker-containers INFO     Module train does not provide a setup.py. 
algo-1-i0u2q_1  | Generating setup.py
algo-1-i0u2q_1  | 2020-06-09 07:11:58,561 sagemaker-containers INFO     Generating setup.cfg
algo-1-i0u2q_1  | 2020-06-09 07:11:58,561 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-i0u2q_1  | 2020-06-09 07:11:58,562 sagemaker-containers INFO     Installing

In [ ]:
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

class JSONPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(JSONPredictor, self).__init__(endpoint_name, sagemaker_session, json_serializer, json_deserializer)

In [14]:
from sagemaker.pytorch import PyTorchModel

training_job_name = estimator.latest_training_job.name
desc = sagemaker_session.sagemaker_client.describe_training_job(TrainingJobName=training_job_name)
trained_model_location = desc['ModelArtifacts']['S3ModelArtifacts']
print(trained_model_location)
model = PyTorchModel(model_data=trained_model_location,
                     role=role,
                     framework_version='1.5.0',
                     entry_point='generate.py',
                     source_dir='code',
                     predictor_cls=JSONPredictor)

s3://sagemaker-us-west-2-294038372338/pytorch-training-2020-06-09-01-27-57-828/output/model.tar.gz


In [15]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Get the end point
endpoint = predictor.endpoint
print(endpoint)

---------------!pytorch-inference-2020-06-09-01-45-36-632


In [16]:
#FIXME: This works, but the predictor using endpoint does not work
input = {
    'seed': 111,
    'temperature': 2.0,
    'words': 100
}
response = predictor.predict(input)
print(response)

allotropes Park Asians floridana carpenteri — Swollen corrupting Ethnographer levels assumed Wróbel
Trends Timberwork camel Business episcopate eh Wigner nitride derrick Antium Cotentin Erna
Lincei extremely XI dysfunctional Canciones was Albisaurus revising Moltke NVA 93156 Chatterji
1240s petroleum falling Celio Alternatively Boveri WAC Mike medial bhasya Bohannon infiltration
plums popular Abruptly Seattle Ravenscar Classically Very Fayella Arcelus Pangani Beltrami Karz
sensation Purdon inherited tireless Penfold Artefacts progeny Tsuga Personally [ Exercises Chesave
Préval Dentzel Computing deceiving Pasternak underestimate Newman Meret demyelinating Crop transcendentalism Bust
reported unstoppable Gambian catalyse 1550 Rivers 767s 1045 prothesis Castillo Communism Willamette
rustics courting Sakanyi Clover 


In [17]:
sagemaker_session.delete_endpoint(predictor.endpoint)